In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import pandas as pd

# Import the TCGA cancer data

In [92]:
from TCGAMultiOmics.multiomics import MultiOmicsData

folder_path = "/home/jonny_admin/PycharmProjects/Bioinformatics_ExternalData/tcga-assembler/LUAD"
external_data_path = "/home/jonny_admin/PycharmProjects/Bioinformatics_ExternalData/"
luad_data = MultiOmicsData(cancer_type="LUAD", tcga_data_path=folder_path, external_data_path=external_data_path,
                           modalities=[\
                                       "GE",
                                       "MIR",
                                       "LNC",
                                       # "CNV",
                                       # "SNP",
#                                        "PRO"
                                       ])



/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


PATIENTS (522, 5)
DRUGS (461, 4)
GE (576, 20472)
MIR (494, 1870)
LNC (546, 12727)
SAMPLES (599, 6)


In [93]:
from moge.utils.file_io import write_node_labels_to_file
write_node_labels_to_file("moge/data/lncRNA_miRNA_mRNA/node_labels.tsv", network, luad_data)

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:450: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  def __init__(self, cancer_type, folder_path):
/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:275: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info.index = self.get_genes_list()
/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6336: FutureWarning: 'Gene Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  rsuffix=rsuffix, sort=sort)


In [115]:
from moge.utils.omics_distance import gower_distance
gower_distance(luad_data.MIR.get_genes_info()[["Mature sequence"]][0:200])

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:450: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  def __init__(self, cancer_type, folder_path):


Gower's dissimilarity: Computing Mature sequence
sequence data similarity 
 count    11476.000000
mean         0.540846
std          0.079644
min          0.227273
25%          0.500000
50%          0.545455
75%          0.590909
max          0.956522
dtype: float64


/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/omics_distance.py:86: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


array([       nan,        nan,        nan, ..., 0.13636364, 0.40909091,
       0.31818182])

In [108]:
max(len("AFDF"), len("DFDSF"))

5

# Heterogeneous Network

In [94]:
from moge.network.heterogeneous_network import HeterogeneousNetwork
from moge.utils.omics_distance import compute_expression_correlations, compute_annotation_similarity
network = HeterogeneousNetwork(modalities=["MIR", "LNC", "GE"], multi_omics_data=luad_data)


MIR  nodes: 1870
LNC  nodes: 12727
GE  nodes: 20472
Total nodes: 35069


In [100]:
nx.write_gml(network.G, "moge/data/lncRNA_miRNA_mRNA/graph.gml")

# Add Node Similarity Edges

In [20]:
network.add_edges_from_nodes_similarity(modality="GE", similarity_threshold=0.7)

/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/omics_distance.py:84: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


700212 edges added.


In [21]:
network.add_edges_from_nodes_similarity(modality="MIR", similarity_threshold=0.7)

Index(['miR family', 'MiRBase ID', 'Seed+m8', 'Mature sequence',
       'Family Conservation?', 'MiRBase Accession'],
      dtype='object')


/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:438: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")


1012 edges added.


/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/omics_distance.py:84: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


In [22]:
network.add_edges_from_nodes_similarity(modality="LNC", similarity_threshold=0.7)

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:262: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.HGNC_lncrna_info.groupby("symbol").first(), on="symbol",
/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6336: FutureWarning: 'Gene Name' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  rsuffix=rsuffix, sort=sort)
/home/jonny_admin/PycharmProjects/MultiOmicsGraphEmbedding/moge/utils/omics_distance.py:84: RuntimeWarning: Mean of empty slice
  pdists_mean = np.nanmean(np.array(individual_variable_distances), axis=0)


26998330 edges added.


In [24]:
network.G.node

{'hsa-let-7a-1': {'modality': 'MIR'},
 'hsa-let-7a-2': {'modality': 'MIR'},
 'hsa-let-7a-3': {'modality': 'MIR'},
 'hsa-let-7b': {'modality': 'MIR'},
 'hsa-let-7c': {'modality': 'MIR'},
 'hsa-let-7d': {'modality': 'MIR'},
 'hsa-let-7e': {'modality': 'MIR'},
 'hsa-let-7f-1': {'modality': 'MIR'},
 'hsa-let-7f-2': {'modality': 'MIR'},
 'hsa-let-7g': {'modality': 'MIR'},
 'hsa-let-7i': {'modality': 'MIR'},
 'hsa-mir-1-1': {'modality': 'MIR'},
 'hsa-mir-1-2': {'modality': 'MIR'},
 'hsa-mir-100': {'modality': 'MIR'},
 'hsa-mir-101-1': {'modality': 'MIR'},
 'hsa-mir-101-2': {'modality': 'MIR'},
 'hsa-mir-103a-1': {'modality': 'MIR'},
 'hsa-mir-103a-2': {'modality': 'MIR'},
 'hsa-mir-103b-1': {'modality': 'MIR'},
 'hsa-mir-103b-2': {'modality': 'MIR'},
 'hsa-mir-105-1': {'modality': 'MIR'},
 'hsa-mir-105-2': {'modality': 'MIR'},
 'hsa-mir-106a': {'modality': 'MIR'},
 'hsa-mir-106b': {'modality': 'MIR'},
 'hsa-mir-107': {'modality': 'MIR'},
 'hsa-mir-10a': {'modality': 'MIR'},
 'hsa-mir-10b': {

# Add Regulatory Interaction edges

In [95]:
# Adds miRNA-target interaction network
network.add_edges_from_edgelist(edgelist=luad_data.MIR.get_miRNA_target_interaction_edgelist(), 
                                modalities=["MIR", "GE"])

Adding edgelist with 284 total unique MIR genes (source), but only matching 247 nodes
Adding edgelist with 13035 total unique GE genes (target), but only matching 12234 nodes


In [96]:
# Adds Gene Regulatory Network edges
network.add_edges_from_edgelist(edgelist=luad_data.GE.get_RegNet_GRN_edgelist(),
                               modalities=["GE", "GE"])

Adding edgelist with 2072 total unique GE genes (source), but only matching 1426 nodes
Adding edgelist with 23158 total unique GE genes (target), but only matching 17694 nodes


In [97]:
# Adds miRNA-lncRNA interaction network
network.add_edges_from_edgelist(edgelist=luad_data.LNC.get_starBase_lncRNA_miRNA_interactions_edgelist(),
                               modalities=["LNC", "MIR"])
network.add_edges_from_edgelist(edgelist=luad_data.LNC.get_lncRNome_miRNA_binding_sites_edgelist(),
                               modalities=["LNC", "MIR"])

Adding edgelist with 1127 total unique LNC genes (source), but only matching 589 nodes
Adding edgelist with 275 total unique MIR genes (target), but only matching 238 nodes
Adding edgelist with 66 total unique LNC genes (source), but only matching 31 nodes
Adding edgelist with 978 total unique MIR genes (target), but only matching 893 nodes


# Add Correlation edges

In [ ]:
mir_ge_corr = network.compute_multiomics_correlations(modalities=["MIR", "GE"])

## Network Info & Visualizations

In [42]:
print("isolated nodes:", len(nx.isolates(network.get_subgraph(modalities=["MIR", "GE", "LNC"]))),
      ", out of total nodes:", network.all_nodes.__len__())
print("edges: ", network.get_subgraph(modalities=["MIR", "GE", "LNC"]).number_of_edges())

isolated nodes:  15293 , out of total nodes: 35069
edges:  363895


In [76]:
print(len(network.G.nodes()))
print(len(network.G.edges()))
# print(luad_data.LNC.get_genes_list())

41338
519546


In [47]:
# Scatter plot of original graph adjacency matrix
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
plt.imshow(compute_annotation_similarity(luad_data.LNC.get_genes_info(), "LNC"), interpolation='nearest', cmap=plt.cm.bwr)
plt.colorbar()
plt.show()

/opt/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:262: FutureWarning: 'symbol' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.gene_info = self.gene_info.join(self.HGNC_lncrna_info.groupby("symbol").first(), on="symbol",


ValueError: columns overlap but no suffix specified: Index(['name', 'location', 'ensembl_gene_id'], dtype='object')

<Figure size 1080x1080 with 0 Axes>

# Laplacian Matrix

In [ ]:
import community
G = nx.read_edgelist('moge/data/karate.edgelist', create_using=nx.Graph())


In [ ]:
A = nx.adjacency_matrix(G)
A.setdiag(A.diagonal()*2)
A = A.todense()

In [ ]:
# -np.sqrt(np.sum(A, axis=1).A1)
# D = np.diag(-np.sqrt(np.sum(A, axis=1).A1))
D = np.diag(np.sum(A, axis=1).A1)

In [ ]:
L = D-A

In [ ]:
print(nx.incidence_matrix(G, oriented=True).shape)
pd.DataFrame(nx.incidence_matrix(G, oriented=True)*nx.incidence_matrix(G, oriented=True).todense().T)

In [ ]:
pd.DataFrame(nx.laplacian_matrix(G).todense())

In [ ]:
from scipy.linalg import decomp_svd
U, s, Vh = decomp_svd.svd(nx.laplacian_matrix(G).todense())

In [ ]:
U[0,:].shape

In [ ]:
partition = community.best_partition(G)
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
# Scatter plot of original graph adjacency matrix
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
plt.imshow(U, interpolation='nearest', cmap=plt.cm.bwr)
plt.colorbar()
plt.show()

# Import Graph Embedding Methods

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

from gem.utils import graph_util, plot_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as gr
from gem.evaluation.metrics import getPrecisionReport
from gem.evaluation.evaluate_link_prediction import evaluateStaticLinkPrediction

from time import time

from gem.embedding.gf       import GraphFactorization
from gem.embedding.hope     import HOPE
from gem.embedding.lap      import LaplacianEigenmaps
from gem.embedding.lle      import LocallyLinearEmbedding
from gem.embedding.node2vec import node2vec
from gem.embedding.sdne     import SDNE
from moge.embedding.dual_graph_embedding import DualGraphEmbedding


# File that contains the edges. Format: source target
# Optionally, you can add weights as third column: source target weight

# Specify whether the edges are directed
isDirected = True

# Load graph
# G = graph_util.loadGraphFromEdgeListTxt(edge_f, directed=isDirected)
# G = G.to_directed()

# G = nx.read_edgelist('../nuclei-segmentation/notebooks/gem/data/ppi.edgelist', create_using=nx.DiGraph())
G = nx.read_edgelist('moge/data/karate.edgelist', create_using=nx.DiGraph())

# G = network.G

# G = nx.convert_node_labels_to_integers(G)

In [ ]:
models = []
# You can comment out the methods you don't want to run
# models.append(HOPE(d=100, beta=0.1))
models.append(GraphFactorization(d=200, max_iter=100, eta=1*10**-4, regu=1.0))
# models.append(LaplacianEigenmaps(d=2))
# models.append(LocallyLinearEmbedding(d=100))
# models.append(node2vec(d=100, max_iter=1, walk_len=80, num_walks=10, con_size=10, ret_p=1, inout_p=1))
models.append(SDNE(d=200, beta=5, alpha=1e-5, nu1=1e-6, nu2=1e-6, 
                   K=3, n_units=[50, 15,], rho=0.3, 
                   n_iter=100, xeta=0.01, n_batch=500))

models.append(DualGraphEmbedding(d=100, reg=1.0, lr=0.05, iterations=100))
                   

In [ ]:
print ('Num nodes: %d, num edges: %d' % (G.number_of_nodes(), G.number_of_edges()))
for embedding in models:
    try:
        print("\n", embedding.get_method_name(), embedding.get_method_summary())
        t1 = time()
        
    #     Learn embedding - accepts a networkx graph or file with edge list
#         embedding.learn_embedding(graph=G, edge_f=None, is_weighted=True, no_python=True)
#         print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))

#         Y = embedding.get_embedding()

#         # Evaluate on graph reconstruction
#         MAP, prec_curv, err, err_baseline = gr.evaluateStaticGraphReconstruction(G, embedding, Y, None)
#         print ("Graph Reconstruction. MAP:", MAP)

        
        # Evaluate on link prediction
        MAP, prec_curv = evaluateStaticLinkPrediction(G, embedding,
                                     train_ratio=0.8,
                                     is_undirected=False)
        print ("Link Prediction. MAP:", MAP)
        print (embedding._method_name+':\n\tTraining time: %f' % (time() - t1))
        
        # Visualize
#         viz.plot_embedding2D(embedding.get_embedding(), di_graph=G, node_colors=None)
#         plt.show()
    except Exception as e:
        print("could not run", embedding, e)

# Plot Adjacency Matrix

In [ ]:
# Scatter plot of original graph adjacency matrix
graph_adj = nx.laplacian_matrix(G.to_undirected()).todense()
plt.figure(figsize=(9,9))
plt.imshow(graph_adj, interpolation='nearest', cmap=plt.cm.gray)
plt.colorbar()
plt.show()

In [ ]:
# Scatter plot of reconstructed adjacency matrix
plt.figure(figsize=(9,9))
plt.imshow(models[0].get_reconstructed_adj(), interpolation='nearest', cmap=plt.cm.gray)
plt.colorbar()
plt.show()

In [ ]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2)
node_pos = model.fit_transform(models[0].get_embedding())
pos = {}
for i in range(len(G.nodes())):
    pos[i] = node_pos[i, :]
nx.draw_networkx(G, pos, width=0.1, node_size=300, arrows=False,
                             alpha=0.8, font_size=12)